# SK: Memory

## Outline
* kernel.memory.save_information_async
* VolatileMemoryStore


## ConversationBufferMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
import semantic_kernel as sk
import os
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

kernel=sk.Kernel()
api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)

# $input is a reserved variable for skills/prompt templates with just one default variable
chatty = kernel.create_semantic_function("{{ $input }}", temperature=0.0)
    


In [4]:
async def chat (instr) :
    # use input_str to pass data into the semantic function $input variable
    output = await kernel.run_async(chatty, input_str=instr)
    return output
    

In [5]:
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(
        "aboutMe", id="info1", text="My name is Andrew"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info2", text="I currently work as a tour guide"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info3", text="I've been living in Seattle since 2005"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info4", text="I visited France and Italy five times since 2015"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info5", text="My family is from New York"
    )

In [6]:
from typing import Tuple
async def setup_chat_with_memory(
    kernel: sk.Kernel,
) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if
    it does not have an answer.

    Information about me, from previous conversations:
    - {{$fact1}} {{recall $fact1}}
    - {{$fact2}} {{recall $fact2}}
    - {{$fact3}} {{recall $fact3}}
    - {{$fact4}} {{recall $fact4}}
    - {{$fact5}} {{recall $fact5}}

    User: {{$user_input}}
    """

    chat_func = kernel.create_semantic_function(
        sk_prompt, max_tokens=200, temperature=0.0
    )

    context = kernel.create_new_context()
    context["fact1"] = "what is my name?"
    context["fact2"] = "where do I live?"
    context["fact3"] = "where's my family from?"
    context["fact4"] = "where have I traveled?"
    context["fact5"] = "what do I do for work?"

    context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = "aboutMe"
    context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8

    return chat_func, context


In [7]:
output=await chat("Hi, My name is Andrew!")
print(output)

Hello Andrew! How can I assist you today?


In [8]:
output=await chat("What is 1+1")
print(output)

1+1 equals 2.


In [10]:
output=await chat("What is my name?")
print(output)
# Without the memory LLM does not know previous info shared

I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation.


In [11]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding
kernel.add_text_embedding_generation_service(
        "ada", OpenAITextEmbedding("text-embedding-ada-002", api_key)
    )

# Give SK some memory to use with the LLM
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())


In [12]:
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7fa808037370>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7fa808037490>}

In [13]:
await populate_memory(kernel)

In [14]:
print("Setting up a chat (with memory!)")
chat_func, context = await setup_chat_with_memory(kernel)

Setting up a chat (with memory!)


In [15]:
context["user_input"] = "What is 1+1?" 
answer = await kernel.run_async(chat_func, input_vars=context.variables)
print(answer)

ChatBot: 1+1 equals 2.


In [16]:
context["user_input"] = "Can you tell my name?" 
answer = await kernel.run_async(chat_func, input_vars=context.variables)
print(answer)

Yes, your name is Andrew.


In [17]:
context["user_input"] = "Can you tell me where I hail from?" 
answer = await kernel.run_async(chat_func, input_vars=context.variables)
print(answer)

You mentioned that your family is from New York.


In [19]:
context["user_input"] = "Can you tell me where I live currently?" 
answer = await kernel.run_async(chat_func, input_vars=context.variables)
print(answer)

You currently live in Seattle.


## ConversationBufferWindowMemory

### User managed in SK for now

## ConversationTokenBufferMemory

### User managed in SK for now

## ConversationSummaryMemory

### User managed in SK for now